<a href="https://colab.research.google.com/github/cuican1432/nlu-naluhodo/blob/master/bert_multi_label_classification_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load github repo of this project [github](https://github.com/cuican1432/nlu-naluhodo)

In [1]:
import sys
!rm -r nlu-naluhodo; rm -r bert_repo
!git clone https://github.com/cuican1432/nlu-naluhodo.git
!cp -r /content/nlu-naluhodo/models/bert-tpu bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

Cloning into 'nlu-naluhodo'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 372 (delta 91), reused 88 (delta 39), pack-reused 222
Receiving objects: 100% (372/372), 10.08 MiB | 16.54 MiB/s, done.
Resolving deltas: 100% (209/209), done.


In [2]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.2.133.226:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 15713879670663694203),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17667312031214739020),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7259243042589462203),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 3063187338904789856),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 2773942055403689470),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 5664809989900726935),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 100087510466400996),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 7264435289128316276),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 876758130747577

In [0]:
import sys
import modeling
import optimization
import run_multilabels_classifier
import tokenization

# import tfhub 
# import tensorflow_hub as hub

import pandas as pd
import numpy as np
import pickle

In [0]:
beeap_1 = pd.read_csv('/content/nlu-naluhodo/data/beeapfinal/beeap_1.csv')
beeap_1.drop('Unnamed: 0', axis=1, inplace=True)
for i in beeap_1.columns[1:]:
    beeap_1[i] = beeap_1[i].astype(np.float64, errors='ignore')
    
data_index = pickle.load(open('/content/nlu-naluhodo/data/data_upsample_index.p', 'rb'))
index = [i for i in range(1702)]
val_idx = np.unique(data_index['val'][0])
test_idx = np.unique(data_index['test'])

train_exp = np.concatenate([test_idx.flatten(),val_idx.flatten()], axis=0)

train_idx = [i for i in index if i not in train_exp]

# train_idx = np.concatenate([train_idx.flatten(),val_idx.flatten()], axis=0)
beeap_1.iloc[train_idx][['Contents','1', '2', '3', '4', '5', '6']].to_csv('/content/nlu-naluhodo/data/train.csv', index = False)
beeap_1.iloc[val_idx][['Contents','1', '2', '3', '4', '5', '6']].to_csv('/content/nlu-naluhodo/data/val.csv', index = False)
beeap_1.iloc[data_index['test']][['Contents','1', '2', '3', '4', '5', '6']].to_csv('/content/nlu-naluhodo/data/test.csv', index = False)


test = beeap_1.iloc[data_index['test']][['Contents','1', '2', '3', '4', '5', '6']].values

In [0]:
# beeap_1 = pd.read_csv('/content/nlu-naluhodo/data/beeapfinal/beeap_1.csv')
# data_index = pickle.load(open('/content/nlu-naluhodo/data/data_upsample_index.p', 'rb'))

# train_sample = data_index['train'][0][0:500]
# val_sample = data_index['val'][0][0:100]


# beeap_1.drop('Unnamed: 0', axis=1, inplace=True)
# for i in beeap_1.columns[1:]:
#     beeap_1[i] = beeap_1[i].astype(np.float64, errors='ignore')
    



# beeap_1.iloc[train_sample][['Contents','1', '2', '3', '4', '5', '6']].to_csv('/content/nlu-naluhodo/data/train.csv', index = False)
# beeap_1.iloc[val_sample][['Contents','1', '2', '3', '4', '5', '6']].to_csv('/content/nlu-naluhodo/data/val.csv', index = False)
# beeap_1.iloc[data_index['test']][['Contents','1', '2', '3', '4', '5', '6']].to_csv('/content/nlu-naluhodo/data/test.csv', index = False)

In [6]:
TASK = 'MultiLabelText-1' #@param {type:"string"}

TASK_DATA_DIR = '/content/nlu-naluhodo/data/'
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
!ls $TASK_DATA_DIR

BUCKET = 'nlu-naluhodo-bert' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
!gsutil ls $BERT_PRETRAINED_DIR
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

***** Task data directory: /content/nlu-naluhodo/data/ *****
beeapfinal  classes.txt  data_upsample_index.p	test.csv  train.csv  val.csv
***** Model output directory: gs://nlu-naluhodo-bert/bert-tfhub/models/MultiLabelText-1 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt


In [7]:
# Model Hyper Parameters
NUM_TRAIN_EPOCHS = 400.0
MAX_SEQ_LENGTH = 256

TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 32
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 3e-6
WARMUP_PROPORTION = 0.01
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
ITERATIONS_PER_LOOP = 1000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

processors = {
  "cola": run_multilabels_classifier.ColaProcessor,
  "mnli": run_multilabels_classifier.MnliProcessor,
  "mrpc": run_multilabels_classifier.MrpcProcessor,
  "multilabeltext-2": run_multilabels_classifier.MultiLabelTextProcessor_original,
  "multilabeltext-1": run_multilabels_classifier.MultiLabelTextProcessor_original
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_multilabels_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=True,
    use_one_hot_embeddings=True
)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=True,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE,
    predict_batch_size = PREDICT_BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': 'gs://nlu-naluhodo-bert/bert-tfhub/models/MultiLabelText-1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.2.133.226:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9ef620b898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.2.133.226:8470', '_evaluation_master': 'grpc://10.2.133.226:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num

In [0]:
# Train the model.
print('multilabel-task on BERT base model normally takes about 10-15 minutes. Please wait...')
train_file = os.path.join(OUTPUT_DIR, "train.tf_record")
run_multilabels_classifier.file_based_convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer, train_file)
tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", TRAIN_BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_multilabels_classifier.file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

multilabel-task on BERT base model normally takes about 10-15 minutes. Please wait...
INFO:tensorflow:Writing example 0 of 1135
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 0
INFO:tensorflow:tokens: [CLS] message id java ##mail evans @ thy ##me date mon mar ps ##t from drew f ##oss ##um @ en ##ron com to darrell school ##craft @ en ##ron com subject t ##w gas sales privileged and confidential attorney client privilege cc danny mcc ##arty @ en ##ron com steven harris @ en ##ron com kevin h ##yat ##t @ en ##ron com mi ##me version content type text plain char ##set us as ##ci ##i content transfer encoding bit bc ##c danny mcc ##arty @ en ##ron com steven harris @ en ##ron com kevin h ##yat ##t @ en ##ron com x from drew f ##oss ##um x to darrell school ##craft x cc danny mcc ##arty steven harris kevin h ##yat ##t x bc ##c x folder drew f ##oss ##um dec june notes folder ##s all documents x origin f ##oss ##um d x file ##name d ##fo ##ss ##um ns ##f in anticipation of potential l

In [0]:
# Eval the model.
eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
num_actual_eval_examples = len(eval_examples)
use_tpu = True
# if use_tpu:
#     # TPU requires a fixed batch size for all batches, therefore the number
#     # of examples must be a multiple of the batch size, or else examples
#     # will get dropped. So we pad with fake examples which are ignored
#     # later on. These do NOT count towards the metric (all tf.metrics
#     # support a per-instance weight, and these get a weight of 0.0).
#     while len(eval_examples) % EVAL_BATCH_SIZE != 0:
#         eval_examples.append(run_multilabels_classifier.PaddingInputExample())

eval_file = os.path.join(OUTPUT_DIR, "eval.tf_record")
run_multilabels_classifier.file_based_convert_examples_to_features(
    eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer, eval_file)

tf.logging.info("***** Running evaluation *****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                len(eval_examples), num_actual_eval_examples,
                len(eval_examples) - num_actual_eval_examples)
tf.logging.info("  Batch size = %d", EVAL_BATCH_SIZE)

# This tells the estimator to run through the entire set.
eval_steps = int(len(eval_examples) // EVAL_BATCH_SIZE)
# However, if running eval on the TPU, you will need to specify the
# number of steps.

eval_drop_remainder = True if use_tpu else False
eval_input_fn = run_multilabels_classifier.file_based_input_fn_builder(
    input_file=eval_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=eval_drop_remainder)

result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys()):
        tf.logging.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

In [0]:
# Predict by the model.
predict_examples = processor.get_test_examples(TASK_DATA_DIR)
num_actual_predict_examples = len(predict_examples)

# if use_tpu:
#     # TPU requires a fixed batch size for all batches, therefore the number
#     # of examples must be a multiple of the batch size, or else examples
#     # will get dropped. So we pad with fake examples which are ignored
#     # later on.
#     while len(predict_examples) % PREDICT_BATCH_SIZE != 0:
#         predict_examples.append(run_multilabels_classifier.PaddingInputExample())

predict_file = os.path.join(OUTPUT_DIR, "predict.tf_record")
run_multilabels_classifier.file_based_convert_examples_to_features(predict_examples, label_list,
                                        MAX_SEQ_LENGTH, tokenizer,
                                        predict_file)

tf.logging.info("***** Running prediction*****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                len(predict_examples), num_actual_predict_examples,
                len(predict_examples) - num_actual_predict_examples)
tf.logging.info("  Batch size = %d", PREDICT_BATCH_SIZE)

predict_drop_remainder = True if use_tpu else False
predict_input_fn = run_multilabels_classifier.file_based_input_fn_builder(
    input_file=predict_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=predict_drop_remainder)


result = estimator.predict(input_fn=predict_input_fn)
output_predict_file = os.path.join(OUTPUT_DIR, "test_results.tsv")

pred = []
with tf.gfile.GFile(output_predict_file, "w") as writer:
    num_written_lines = 0
    tf.logging.info("***** Predict results *****")
    for (i, prediction) in enumerate(result):
        probabilities = prediction["probabilities"]
        pred.append((test[i][0], probabilities, test[i][1:]))
        if i >= num_actual_predict_examples:
            break
        output_line = ",".join(str(class_probability) for class_probability in probabilities) + "\n"
        writer.write(str(predict_examples[i].guid) + ',' + output_line)
        num_written_lines += 1
        
pickle.dump(pred, open("pred.p", "wb"))

In [0]:
pred[:5]

In [0]:
cd /content/nlu-naluhodo

In [0]:
!mv data/train.csv  data/test.csv

In [0]:
# Predict by the model by training data.
test = beeap_1.iloc[train_idx][['Contents','1', '2', '3', '4', '5', '6']].values
predict_examples = processor.get_test_examples(TASK_DATA_DIR)
num_actual_predict_examples = len(predict_examples)

# if use_tpu:
#     # TPU requires a fixed batch size for all batches, therefore the number
#     # of examples must be a multiple of the batch size, or else examples
#     # will get dropped. So we pad with fake examples which are ignored
#     # later on.
#     while len(predict_examples) % PREDICT_BATCH_SIZE != 0:
#         predict_examples.append(run_multilabels_classifier.PaddingInputExample())

predict_file = os.path.join(OUTPUT_DIR, "predict.tf_record")
run_multilabels_classifier.file_based_convert_examples_to_features(predict_examples, label_list,
                                        MAX_SEQ_LENGTH, tokenizer,
                                        predict_file)

tf.logging.info("***** Running prediction*****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                len(predict_examples), num_actual_predict_examples,
                len(predict_examples) - num_actual_predict_examples)
tf.logging.info("  Batch size = %d", PREDICT_BATCH_SIZE)

predict_drop_remainder = True if use_tpu else False
predict_input_fn = run_multilabels_classifier.file_based_input_fn_builder(
    input_file=predict_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=predict_drop_remainder)


result = estimator.predict(input_fn=predict_input_fn)
output_predict_file = os.path.join(OUTPUT_DIR, "test_results.tsv")

pred_train = []
with tf.gfile.GFile(output_predict_file, "w") as writer:
    num_written_lines = 0
    tf.logging.info("***** Predict results *****")
    for (i, prediction) in enumerate(result):
        probabilities = prediction["probabilities"]
        pred_train.append((test[i][0], probabilities, test[i][1:]))
        if i >= num_actual_predict_examples:
            break
        output_line = ",".join(str(class_probability) for class_probability in probabilities) + "\n"
        writer.write(str(predict_examples[i].guid) + ',' + output_line)
        num_written_lines += 1
        
pickle.dump(pred, open("pred_train.p", "wb"))

In [0]:
from sklearn.metrics import f1_score

from eval.evaluation import pred_classes_dt, pred_classes_f1, plot_roc_curve, \
  plot_confusion_matrix

from sklearn.metrics import f1_score

In [0]:
def extract(pred, k):
  res = []
  for p in pred:
    res.append(p[k])
  return np.array(res, dtype = float)

In [0]:
predictions = extract(pred, 1)
train_pred =  extract(pred_train, 1)
y_train = extract(pred_train, 2)
y_test =  extract(pred, 2)

y_pred_dt = pred_classes_dt(train_pred, y_train, predictions)

In [0]:
y_pred_dt = pred_classes_dt(train_pred, y_train, predictions)
print('macro F1 score: {0:0.4f}'.format(f1_score(y_test, y_pred_dt, average='macro')))
print('micro F1 score: {0:0.4f}'.format(f1_score(y_test, y_pred_dt, average='micro')))


y_pred_f1 = pred_classes_f1(train_pred, y_train,predictions)
print('macro F1 score: {0:0.4f}'.format(f1_score(y_test, y_pred_f1, average='macro')))
print('micro F1 score: {0:0.4f}'.format(f1_score(y_test, y_pred_f1, average='micro')))

In [0]:
plot_roc_curve(y_test, predictions,  title='ROC Curves for Coarse Genre', micro=True, per_class=True)

In [0]:
res = {'y_test': y_test, 'y_test_pred': predictions, 'y_train_pred':train_pred, 'y_train':y_train}

In [0]:
pickle.dump(res, open("bert_result.p", "wb"))

In [0]:
# # Export the model
# def serving_input_fn():
#   with tf.variable_scope("foo"):
#     feature_spec = {
#         "input_ids": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
#         "input_mask": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
#         "segment_ids": tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
#         "label_ids": tf.FixedLenFeature([6], tf.int64),
#       }
#     serialized_tf_example = tf.placeholder(dtype=tf.string,
#                                            shape=[None],
#                                            name='input_example_tensor')
#     receiver_tensors = {'examples': serialized_tf_example}
#     features = tf.parse_example(serialized_tf_example, feature_spec)
#     return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

# EXPORT_DIR = 'gs://{}/bert/export/{}'.format(BUCKET, TASK)
# estimator._export_to_tpu = False  # this is important
# path = estimator.export_savedmodel(EXPORT_DIR, serving_input_fn)
# print(path)


In [0]:
# !saved_model_cli show --all --dir gs://nlu-naluhodo-bert/bert/export/MultiLabelText/1557283389